CATCOMB  -- Catalytic combustion of methane on platinum.

This script solves a catalytic combustion problem. A stagnation flow is set
up, with a gas inlet 10 cm from a platinum surface at 900 K. The lean,
premixed methane/air mixture enters at ~ 6 cm/s (0.06 kg/m2/s), and burns
catalytically on the platinum surface. Gas-phase chemistry is included too,
and has some effect very near the surface.

The catalytic combustion mechanism is from Deutschman et al., 26th
Symp. (Intl.) on Combustion,1996 pp. 1747-1754

In [1]:
import numpy as np
import cantera as ct

In [2]:
#  Parameter values are collected here to make it easier to modify them
p = ct.one_atm  # pressure
tinlet = 300.0  # inlet temperature
tsurf = 900.0  # surface temperature
mdot = 0.06  # kg/m^2/s
transport = 'Mix'  # transport model

In [3]:
# We will solve first for a hydrogen/air case to use as the initial estimate
# for the methane/air case

# composition of the inlet premixed gas for the hydrogen/air case
comp1 = 'H2:0.05, O2:0.21, N2:0.78, AR:0.01'

# composition of the inlet premixed gas for the methane/air case
comp2 = 'CH4:0.095, O2:0.21, N2:0.78, AR:0.01'

# The inlet/surface separation is 10 cm.
width = 0.1 # m

loglevel = 1  # amount of diagnostic output (0 to 5)

In [4]:
################ create the gas object ########################
#
# This object will be used to evaluate all thermodynamic, kinetic, and
# transport properties. The gas phase will be taken from the definition of
# phase 'gas' in input file 'ptcombust.cti,' which is a stripped-down version
# of GRI-Mech 3.0.
gas = ct.Solution('ptcombust.cti', 'gas')
gas.TPX = tinlet, p, comp1

In [5]:
################ create the interface object ##################
#
# This object will be used to evaluate all surface chemical production rates.
# It will be created from the interface definition 'Pt_surf' in input file
# 'ptcombust.cti,' which implements the reaction mechanism of Deutschmann et
# al., 1995 for catalytic combustion on platinum.
#
surf_phase = ct.Interface('ptcombust.cti', 'Pt_surf', [gas])
surf_phase.TP = tsurf, p

In [6]:
# integrate the coverage equations in time for 1 s, holding the gas
# composition fixed to generate a good starting estimate for the coverages.
surf_phase.advance_coverages(1.0)

# create the object that simulates the stagnation flow, and specify an initial
# grid
sim = ct.ImpingingJet(gas=gas, width=width, surface=surf_phase)

In [7]:
# Objects of class StagnationFlow have members that represent the gas inlet
# ('inlet') and the surface ('surface'). Set some parameters of these objects.
sim.inlet.mdot = mdot
sim.inlet.T = tinlet
sim.inlet.X = comp1
sim.surface.T = tsurf

# Show the initial solution estimate
sim.show_solution()



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> inlet <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Mass Flux:         0.06 kg/m^2/s 
    Temperature:        300 K 
    Mass Fractions: 
                      H2    0.003467 
                      O2      0.2312 
                      AR     0.01374 
                      N2      0.7516 



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> flame <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Pressure:   1.013e+05 Pa

-------------------------------------------------------------------------------
          z           u           V           T      lambda      eField 
-------------------------------------------------------------------------------
          0     0.05335           0         300           0           0 
       0.02     0.04268           0         420           0           0 
       0.04     0.03201           0         540           0           0 
       0.06     0.02134           0         660           0           0 
       0.08     0.01067           0         780 

In [8]:
# Solving problems with stiff chemistry coulpled to flow can require a
# sequential approach where solutions are first obtained for simpler problems
# and used as the initial guess for more difficult problems.

# disable the surface coverage equations, and turn off all gas and surface
# chemistry.
sim.surface.coverage_enabled = False
surf_phase.set_multiplier(0.0)
gas.set_multiplier(0.0)

# solve the problem, refining the grid if needed, to determine the non-
# reacting velocity and temperature distributions
sim.solve(loglevel, auto=True)


************ Solving on 6 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [6] point grid(s).

..............................................................................
grid refinement disabled.

******************** Solving with grid refinement enabled ********************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [6] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 3 4 
    to resolve T V 
##############################################################################

..........................................

In [9]:
# now turn on the surface coverage equations, and turn the chemistry on slowly
sim.surface.coverage_enabled = True
for mult in np.logspace(-5, 0, 6):
    surf_phase.set_multiplier(mult)
    gas.set_multiplier(mult)
    print('Multiplier =', mult)
    sim.solve(loglevel)

# At this point, we should have the solution for the hydrogen/air problem.
sim.show_solution()

Multiplier = 1e-05

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [10] point grid(s).

..............................................................................
no new points needed in flame
Multiplier = 0.0001

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [10] point grid(s).

..............................................................................
no new points needed in flame
Multiplier = 0.001

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [10] point grid(s).

..............................................................................
no new points needed in flame
Multiplier = 0.01

...............................................

In [10]:
# Now switch the inlet to the methane/air composition.
sim.inlet.X = comp2

# set more stringent grid refinement criteria
sim.set_refine_criteria(100.0, 0.15, 0.2, 0.0)

# solve the problem for the final time
sim.solve(loglevel)

# show the solution
sim.show_solution()



..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003844    -0.8978
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01478    -0.9305
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.3788    -0.2857
Attempt Newton solution of steady-state problem...    success.

Problem solved on [10] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 0 1 2 3 4 5 6 7 8 
    to resolve AR C2H6 CH2O CH4 CO CO2 H2O N2 O2 OH T V u 
##############################################################################

..............................................................................
Attempt Newton solution of steady-s

In [11]:
# save the solution in XML format. The 'restore' method can be used to restart
# a simulation from a solution stored in this form.
sim.save("catcomb.xml", "soln1")

# save selected solution components in a CSV file for plotting in
# Excel or MATLAB.
sim.write_csv('catalytic_combustion.csv', quiet=False)

sim.show_stats(0)

Solution saved to file catcomb.xml as solution soln1.
Solution saved to 'catalytic_combustion.csv'.

Statistics:

 Grid   Timesteps  Functions      Time  Jacobians      Time
    8           0         84        NA          1        NA
   10           0          6        NA          1        NA
   12           0          6        NA          1        NA
   12           0         18        NA          1        NA
   12           0          4        NA          1        NA
   12           0         10        NA          1        NA
   12           0         18        NA          1        NA
   12           0         14        NA          1        NA
   12           0          6        NA          1        NA
   12          30        271        NA         12        NA
   21           0         42        NA          3        NA
   30           0         17        NA          2        NA
   33           0         30        NA          2        NA
   34           0         19        NA        

In [ ]:
# https://cantera.org/examples/python/surface_chemistry/catalytic_combustion.py.html